In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 1000 observations

In [2]:
n_sim, nobs, expo = 100, 1000, 250
err_pnl, popsize = 3e5, 5000

In [3]:
# Setting the true model
f1, f2 = bs.loss_model("Weibull", ["k1", "β1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])

f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

%time PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

CPU times: user 39.8 s, sys: 40.8 ms, total: 39.9 s
Wall time: 39.9 s


array([-173.96936159, -127.04112545,  -54.61721873])

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [4]:
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
def fit_one(k, f, f_name,  prior, X):
    trace, log_marg, DIC, WAIC = bs.smc(X, f, popsize, prior, verbose = False)
    PnLs = np.array(f.PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
    cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
    return([k, f_name , nobs, log_marg, trace["γ"].mean(), parms_true[-1] ] + cap.tolist() + true_cap.tolist())
def fit_all(k):
    print("simulation #"+ str(k))
    col_names = ["sim", "model_name", "nobs", "log_marg", "γ_map", "γ_true", "cap005", "cap01", "cap05", "cap005_true", "cap01_true", "cap05_true" ]
    df_temp = pd.DataFrame([fit_one(k, fs[i], f_names[i], prior_spliced_model[i], Xs[k]) for i in range(len(fs))], columns = col_names)
    df_temp['posterior_probability'] = np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)) / \
    np.sum(np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)))  
    γ_BMA = np.dot(df_temp['posterior_probability'].values, df_temp['γ_map'])
    cap_bma = np.dot(df_temp['posterior_probability'].values, df_temp[["cap005", "cap01", "cap05"]].values)
    BMA_df = pd.DataFrame([[k, "BMA" , nobs, None, γ_BMA, parms_true[-1] ] + cap_bma.tolist() + true_cap.tolist()],
            columns = col_names)
    df_temp = pd.concat([df_temp, BMA_df])
    return(df_temp)
%time res = Parallel(n_jobs= 50)(delayed(fit_all)(k) for k in range(n_sim))
pd.concat(res).to_csv("../../Data/Simulations/simu_wellspec_"+str(nobs)+".csv", sep=',')

CPU times: user 8.8 s, sys: 617 ms, total: 9.42 s
Wall time: 34min 15s


In [5]:
pd.concat(res)

,sim,model_name,nobs,log_marg,γ_map,γ_true,cap005,cap01,cap05,cap005_true,cap01_true,cap05_true,posterior_probability
0,0,Weibull_Log-Logistic,1000,-811.430314,3.175958,1.5,-122.221779,-79.583618,-19.977254,-173.969362,-127.041125,-54.617219,4.463477e-01
1,0,Weibull_Lomax,1000,-811.857183,2.940089,1.5,-145.951352,-103.714999,-41.983446,-173.969362,-127.041125,-54.617219,2.912636e-01
2,0,Weibull_Inverse-Weibull,1000,-811.961585,2.839506,1.5,-211.620660,-137.253898,-38.851765,-173.969362,-127.041125,-54.617219,2.623886e-01
3,0,Inverse-Weibull_Log-Logistic,1000,-830.144589,2.613552,1.5,-175.393525,-110.269082,-22.169537,-173.969362,-127.041125,-54.617219,3.327873e-09
4,0,Inverse-Weibull_Lomax,1000,-830.435664,2.342440,1.5,-120.475698,-77.367886,-12.784584,-173.969362,-127.041125,-54.617219,2.487452e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,99,Weibull_Inverse-Weibull,1000,-880.531303,3.249164,1.5,-521.224691,-353.324564,-132.383892,-173.969362,-127.041125,-54.617219,1.859836e-01
3,99,Inverse-Weibull_Log-Logistic,1000,-928.86835,0.005468,1.5,-64717.750705,-29497.285878,-5535.553504,-173.969362,-127.041125,-54.617219,1.892179e-22
4,99,Inverse-Weibull_Lomax,1000,-926.589963,0.065517,1.5,-1139.705002,-735.884519,-267.861332,-173.969362,-127.041125,-54.617219,1.846941e-21
5,99,Inverse-Weibull_Inverse-Weibull,1000,-939.729337,1.980682,1.5,-614.575636,-387.372085,-111.115030,-173.969362,-127.041125,-54.617219,3.631575e-27
